Este conjunto de datos se utiliza para la segmentación de fases de gestos en videos grabados con un sensor Microsoft Kinect. El dataset ya ha sido preprocesado, con los archivos conteniendo información tanto en forma de posiciones espaciales (coordenadas X, Y, Z) de puntos de articulación clave (manos, muñeca, cabeza y columna) como de velocidades y aceleraciones vectoriales y escalares de las extremidades.

El objetivo principal de este dataset es predecir las fases de gestos en una secuencia temporal, categorizando cada momento de la gesticulación en una de las siguientes clases: Descanso (D), Preparación (P), Golpe (S), Sujeción (H) y Retracción (R). Estas fases corresponden a diferentes etapas de un gesto, como el inicio, el movimiento y el retorno del gesto. El desafío es utilizar esta información para entrenar modelos que puedan segmentar correctamente los gestos a partir de las posiciones y movimientos de las extremidades, lo que se traduce en una clasificación precisa de cada fase en un video.

Estos videos fueron segmentados manualmente por expertos, lo que proporciona un conjunto de ground truth necesario para entrenar y evaluar modelos de predicción.

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import optuna
from sklearn.metrics import f1_score

ModuleNotFoundError: No module named 'tensorflow.python'

In [2]:
folder_path = "C:/Users/Finar/One_finarq_staf/OneDrive/Documentos/Proyecto Modelos/proyecto-modelos/data/Classification"

In [13]:
csv_files = [f for f in os.listdir(folder_path) if f.endswith('_va3.csv')]

In [14]:
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)


In [15]:
df = pd.concat(dfs, ignore_index=True)

In [16]:
df

,1,2,3,4,5,6,7,8,9,10,...,24,25,26,27,28,29,30,31,32,Phase
0,-0.005009,-0.000964,0.000573,0.008623,0.005667,0.001302,-0.000631,0.000130,-0.000048,0.007762,...,1.880800e-04,0.005133,0.010400,0.000646,0.007871,0.004631,0.000963,0.000092,0.000438,D
1,0.004905,0.001209,-0.000649,0.004737,0.003166,0.000819,-0.000572,-0.000015,0.000023,0.002706,...,-7.500000e-07,0.005093,0.005756,0.000573,0.003459,0.000730,0.000332,0.000012,0.000433,D
2,-0.002393,-0.000216,0.000136,0.003028,0.001212,0.000336,-0.000449,0.000017,0.000047,0.002868,...,-3.920000e-05,0.002406,0.003279,0.000452,0.003261,0.002412,0.000852,0.000042,0.000202,D
3,-0.001394,-0.000242,0.000056,0.001182,0.000575,0.000225,-0.000479,-0.000050,0.000104,0.001171,...,-3.184000e-05,0.001416,0.001334,0.000493,0.001358,0.000313,0.000611,0.000029,0.000596,D
4,-0.000156,-0.000004,0.000023,0.001585,0.000630,0.000094,-0.000303,0.000097,0.000065,0.001579,...,-2.015000e-05,0.000158,0.001709,0.000325,0.001713,0.000203,0.000069,0.000038,0.000069,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9868,-0.003709,-0.006168,0.000786,-0.000155,0.001088,-0.000144,-0.003815,-0.004658,0.000656,0.000060,...,-9.400000e-07,0.007240,0.001109,0.006056,0.000173,0.001976,0.000137,0.001575,0.000051,D
9869,-0.000727,0.001536,-0.000211,0.000700,-0.000975,0.000067,-0.001147,0.000177,0.000008,0.000423,...,1.234000e-05,0.001712,0.001202,0.001160,0.000515,0.002600,0.000701,0.001738,0.000124,D
9870,0.003074,0.007870,-0.000962,0.000526,-0.000779,0.000090,0.002786,0.005035,-0.000606,0.000413,...,2.327000e-05,0.008504,0.000944,0.005786,0.000431,0.002007,0.000071,0.001697,0.000112,D
9871,0.003297,0.008467,-0.001035,0.000578,-0.000740,0.000101,0.002947,0.005385,-0.000652,0.000580,...,1.541000e-05,0.009145,0.000944,0.006173,0.000699,0.000229,0.000023,0.000139,0.000113,D


In [18]:
df.isnull().sum().sum()

np.int64(0)

In [20]:
#Separar en X and Y 
X = df.drop('Phase', axis=1) 
Y = df['Phase']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'train_test_split' is not defined

In [ ]:
#KNN
knn_model = KNeighborsClassifier(n_neighbors = 5)
knn_model.fit(X_train, y_train)

y_pred_knn = knn_model.predict(X_test)

print("k-Nearest Neighbors (KNN) Classification Report:")
print(classification_report(y_test, y_pred_knn))
print("=" * 50)